In [1]:
import torch
from dak.kernels.laplace_kernel import LaplaceProductKernel
from dak.utils.sparse_design.design_class import HyperbolicCrossDesign, SparseGridDesign
from dak.utils.operators.chol_inv import mk_chol_inv, tmk_chol_inv

In [2]:
dyadic_design = HyperbolicCrossDesign(dyadic_sort=True, return_neighbors=True)(deg=2, input_lb=0, input_ub=1)
chol_inv = mk_chol_inv(
    dyadic_design=dyadic_design,
    markov_kernel=LaplaceProductKernel(lengthscale=1.),
    upper=True)  # [m, m] size tensor
design_points = dyadic_design.points.reshape(-1, 1)  # [m, 1] size tensor

In [3]:
print(chol_inv.to_dense())

tensor([[ 1.0000, -1.2416, -1.2416],
        [ 0.0000,  1.5942,  0.0000],
        [ 0.0000,  0.0000,  1.5942]])


In [4]:
x = torch.Tensor([0.35]).unsqueeze(dim=-1)
h = LaplaceProductKernel(lengthscale=1.)(x, design_points)
out = torch.matmul(h, chol_inv)
print(h)
print(out)

x = torch.Tensor([0.65]).unsqueeze(dim=-1)
h = LaplaceProductKernel(lengthscale=1.)(x, design_points)
out = torch.matmul(h, chol_inv)
print(h)
print(out)
non_zero_indices = torch.nonzero(out, as_tuple=True)
print(out[non_zero_indices])

tensor([[0.8607, 0.9048, 0.6703]])
tensor([[ 8.6071e-01,  3.7387e-01, -1.4539e-07]])
tensor([[0.8607, 0.6703, 0.9048]])
tensor([[ 8.6071e-01, -1.4539e-07,  3.7387e-01]])
tensor([ 8.6071e-01, -1.4539e-07,  3.7387e-01])
